# Housing Prices Competition 

### Objective: Predict home prices in Iowa using 79 explanatory variables describing (almost) every aspect of the homes

In [5]:
# import libraries

import pandas as pd
from sklearn.model_selection import train_test_split

# Load and read the data

X_full = pd.read_csv("C:/Users/EBEN-SUCCESS/Desktop/AIM/Kaggle/Housing Prices Competition/data/train.csv", index_col="Id")

X_test_full = pd.read_csv("C:/Users/EBEN-SUCCESS/Desktop/AIM/Kaggle/Housing Prices Competition/data/test.csv", index_col="Id")

# # Obtain the target and predictors
# y = X_full.SalePrice
# features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
# X = X_full[features].copy()
# X_test = X_test_full[features].copy()

# # Break off validation set from training data
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [6]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

### View the data

In [7]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


### Step 1: Preliminary Investigation

In [8]:
# shape of training data (num_rows, num_column)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


Since the data has few missing entries, dropping columns is unlikely ot yield good result.

### Define five different random forest models

In [3]:
# from sklearn.ensemble import RandomForestRegressor

# # Define the models
# model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
# model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
# model_3 = RandomForestRegressor(n_estimators=100, criterion ='absolute_error', random_state=0)
# model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
# model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

# models = [model_1, model_2, model_3, model_4, model_5]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(no_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Create function called score_model() that selects the best model.

#### This function does so by returning the absolute mean error (MAE)

In [4]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model: %d MAE: %d" %(i+1, mae))

Model: 1 MAE: 24015
Model: 2 MAE: 23740
Model: 3 MAE: 23528
Model: 4 MAE: 23996
Model: 5 MAE: 23706


In [6]:
# Chose the best model
# That is the model with the least MAE

best_model = model_3

Generate Test Predictions

In [7]:
# Define a model
# Here I will use a Random Forest Model

my_model = RandomForestRegressor(random_state=1)

In [10]:
# Fit the model to the training data
my_model.fit(X, y)

# Generate test predictions
pred_test = my_model.predict(X_test)

# Save predictions in format used in competition scoring

output = pd.DataFrame({'Id': X_test.index, 'SalePrice': pred_test})
output.to_csv('submission.csv', index=False)